In [1]:
from keras.datasets import cifar10
import tensorflow as tf

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
x_validation = x_test[0:5000]
y_validation = y_test[0:5000]

x_test = x_test[5000:10000]
y_test = y_test[5000:10000]

num_classes = 10

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(x_validation.shape, y_validation.shape)


(50000, 32, 32, 3) (50000, 1)
(5000, 32, 32, 3) (5000, 1)
(5000, 32, 32, 3) (5000, 1)


In [4]:
# using data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

train_datagen.fit(x_train)

validation_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

validation_datagen.fit(x_validation)

In [5]:
x_train, x_test, x_validation = x_train/255.0, x_test/255.0, x_validation/255.0
y_train, y_test, y_validation = y_train.flatten(), y_test.flatten(), y_validation.flatten()

In [6]:
print(x_train.shape, y_train.shape)

(50000, 32, 32, 3) (50000,)


In [7]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:
            print("\nReached 99% accuracy so stopping training!")
            self.model.stop_training = True
        if logs.get('accuracy') is not None and logs.get('accuracy') < 0.01:
            print("\nVery low accuracy, something is wrong with the model!")
            self.model.stop_training = True

In [8]:
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential

In [9]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.25),
    
    Flatten(),
    Dropout(0.25),
    
    Dense(1024, activation='relu'),
    Dropout(0.25),
    
    Dense(num_classes, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                        

In [10]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [24]:
callbacks = myCallback()

history = model.fit(
    x_train,
    y_train,
    epochs=30,
    validation_data=(
        x_validation,
        y_validation
    ),
    callbacks=[callbacks]
)

Epoch 1/30
1563/1563 [==============================] - 265s 170ms/step - loss: 1.4425 - accuracy: 0.4899 - val_loss: 1.1772 - val_accuracy: 0.5874
Epoch 2/30
1563/1563 [==============================] - 250s 160ms/step - loss: 1.0179 - accuracy: 0.6404 - val_loss: 0.8218 - val_accuracy: 0.7116
Epoch 3/30
1563/1563 [==============================] - 251s 160ms/step - loss: 0.8702 - accuracy: 0.6971 - val_loss: 0.8463 - val_accuracy: 0.7120
Epoch 4/30
1563/1563 [==============================] - 263s 168ms/step - loss: 0.7744 - accuracy: 0.7337 - val_loss: 0.7633 - val_accuracy: 0.7412
Epoch 5/30
1563/1563 [==============================] - 257s 164ms/step - loss: 0.7072 - accuracy: 0.7576 - val_loss: 0.6665 - val_accuracy: 0.7718
Epoch 6/30
1563/1563 [==============================] - 252s 161ms/step - loss: 0.6368 - accuracy: 0.7796 - val_loss: 0.6302 - val_accuracy: 0.7832
Epoch 7/30
1563/1563 [==============================] - 264s 169ms/step - loss: 0.5964 - accuracy: 0.7952 - val_

In [25]:
scores = model.evaluate(x_test, y_test, verbose=1)
print(f'test accuracy is {scores[1]}')

157/157 [==============================] - 6s 39ms/step - loss: 0.4944 - accuracy: 0.8634
test accuracy is 0.8633999824523926


In [26]:
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model\assets


INFO:tensorflow:Assets written to: saved_model/my_model\assets
